In [180]:
from streetlevel import streetview
from config import Config
from src.inference.segment import detect_trees
from src.inference.depth import estimate_depth
from src.utils.unwrap import divide_panorama
from src.utils.masks import add_masks, remove_duplicates, make_image
from src.utils.transformation import get_point
from src.utils.geodesic import get_coordinates
from cli import build_config
from models.DepthAnything.depth_anything_v2.dpt import DepthAnythingV2
from models.CalibrateDepth.model import DepthCalibrator
from config import Config
import torch
from ultralytics import YOLO
import pandas as pd
import folium
import os
import cv2
import numpy as np
import nest_asyncio
from IPython.display import display
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
IO_EXECUTOR = ThreadPoolExecutor(max_workers=4)


In [166]:
# Load the CSV file
df = pd.read_csv('streetviews/chandigarh_28_29.csv')
groundtruth_df = pd.read_csv("eval/28_29_groundtruth.csv")

# Get the center of the map (mean of lat/lng)
center_lat = df['lat'].mean()
center_lng = df['lng'].mean()

# Create a folium map
m = folium.Map(location=[center_lat, center_lng], zoom_start=13, max_zoom=25)

# Add markers for each point
for idx, row in df.iterrows():
    folium.CircleMarker(
        [row['lat'], row['lng']],
        radius=2,
        color='blue',
        fill=True,
        fill_color='blue',
        popup=str(row['pano_id'])
    ).add_to(m)

# Display the map
m


In [167]:
def load_models(config: Config):
    depth_model = DepthAnythingV2(**{**config.MODEL_CONFIGS["vitl"], "max_depth": 80})
    depth_model.load_state_dict(torch.load(config.DEPTH_MODEL_PATH, map_location="cpu"))
    depth_model.to(config.DEVICE).eval()
    tree_model = YOLO(config.TREE_MODEL_PATH)
    depth_calibrator = DepthCalibrator(config.DEPTH_CALIBRATION_MODEL_PATH)
    
    return depth_model, tree_model, depth_calibrator

class MockArgs:
    def __init__(self):
        self.input_csv = 'streetviews/chandigarh_28_29.csv'
        self.output_csv = 'chandigarh_28_29_processed.csv'
        self.fov = 90
        self.width = 1024
        self.height = 720

args = MockArgs()
print(f"Using mock arguments for notebook environment:")
print(f"- Input CSV: {args.input_csv}")
print(f"- Output CSV: {args.output_csv}")
print(f"- FOV: {args.fov}")
print(f"- Width: {args.width}")
print(f"- Height: {args.height}")

config = build_config(args)

Using mock arguments for notebook environment:
- Input CSV: streetviews/chandigarh_28_29.csv
- Output CSV: chandigarh_28_29_processed.csv
- FOV: 90
- Width: 1024
- Height: 720


In [168]:
depth_model, tree_model, depth_calibrator = load_models(config)

In [174]:
nest_asyncio.apply()
pano = streetview.find_panorama_by_id(
    "BBwnrNpjqIYg93rPyvrWJQ", download_depth=True
)
pil_image = streetview.get_panorama(pano)


In [175]:
image = np.array(pil_image)
views = divide_panorama(image, config.HEIGHT, config.WIDTH, config.FOV)

trees = []
for i, (view, theta) in enumerate(views):
    tree_data = detect_trees(view, tree_model, config.DEVICE)
print(len(tree_data))

1


In [176]:
def rescale_point(h, w, orig_h=6656, orig_w=13312, new_h=128, new_w=256):
    """
    Rescale a (h, w) point from original size to new size.
    """
    new_h_coord = h * (new_h / orig_h)
    new_w_coord = w * (new_w / orig_w)
    return int(new_h_coord), int(new_w_coord)

In [ ]:
row = groundtruth_df[groundtruth_df["pano_id"] == pano.id]

trees = []
for j, tree in enumerate(tree_data):
    masks = tree.masks
    boxes = tree.boxes
    if masks is not None:
        for k, mask in enumerate(masks):
            image_path = f"{pano.id}_tree{j}_box{k}.jpg"
            conf = boxes[k].conf.item()
            IO_EXECUTOR.submit(
                make_image, view, boxes[k], mask, image_path
            )
            orig_point, pers_point = get_point(mask, theta, pano, config.HEIGHT, config.WIDTH, config.FOV)
            
            orig_point_h, orig_point_w = rescale_point(row["image_y"].values[0], row["image_x"].values[0])
            orig_point_h_, orig_point_w_ = rescale_point(orig_point[1], orig_point[0])
            pano_distance = pano.depth.data[orig_point_h][orig_point_w]
            pano_distance_ = pano.depth.data[orig_point_h_][orig_point_w_]
            print(pano_distance, pano_distance_)
            lat_pano, lon_pano = get_coordinates(pano, orig_point, image.shape[1], pano_distance)
            plt.figure(figsize=(8, 6))
            plt.imshow(image)
            plt.scatter(orig_point[0], orig_point[1], c='red', marker='o', label='Detected Tree')
            if not row.empty:
                gt_x = row["image_x"].values[0]
                gt_y = row["image_y"].values[0]
                plt.scatter(gt_x, gt_y, c='yellow', marker='x', label='Ground Truth')
            plt.legend()
            plt.title(f"Pano {pano.id} - Tree {j} Box {k}")
            plt.axis('off')
            plt.show()
            tree = {
                    "image_path": image_path,
                    "pano_id": pano.id,
                    "stview_lat": pano.lat,
                    "stview_lng": pano.lon,
                    "tree_lat": lat_pano,
                    "tree_lng": lon_pano,
                    "image_x": float(orig_point[0]),
                    "image_y": float(orig_point[1]),
                    "theta": theta,
                    "mask": mask,
                    "conf": conf,
                    "distance_pano": pano_distance,
                }
            trees.append(tree)

In [ ]:
if trees:
    avg_lat = sum(tree["tree_lat"] for tree in trees) / len(trees)
    avg_lng = sum(tree["tree_lng"] for tree in trees) / len(trees)
else:
    avg_lat = pano.lat
    avg_lng = pano.lon

m = folium.Map(location=[avg_lat, avg_lng], zoom_start=18)

for tree in trees:
    folium.CircleMarker(
        location=[tree["tree_lat"], tree["tree_lng"]],
        popup=f'Conf: {tree["conf"]:.2f}',
        radius=2,
        color='blue',
        fill=True,
        fill_color='blue',
    ).add_to(m)
folium.CircleMarker(
    location=[pano.lat, pano.lon],
    popup=f'Pano {pano.id}',
    radius=2,
    color='red',
    fill=True,
    fill_color='red',
).add_to(m)
folium.CircleMarker(
    location=[row["tree_lat"].values[0], row["tree_lng"].values[0]],
    popup=f'Ground Truth',
    radius=2,
    color='yellow',
    fill=True,
    fill_color='yellow',
).add_to(m)
m

In [109]:
df_part = remove_duplicates(
    pd.DataFrame(trees),
    image.shape[1], image.shape[0],
    config.HEIGHT, config.WIDTH, config.FOV
)
print(len(df_part))

def _save_full(img=image.copy(), td=df_part.copy(), pid=pano.id):
    full = add_masks(img, td, config.HEIGHT, config.WIDTH, config.FOV)
    full_path = os.path.join(f"{pid}.jpg")
    cv2.imwrite(full_path, full)
    
IO_EXECUTOR.submit(_save_full)

1


<Future at 0x160337ee0 state=pending>

In [179]:
pano.lat, pano.lon

(30.71436385688168, 76.79760008586778)